# **HACKATHON AIRBUS HELICOPTERS 2024**

## Gestion des librairies


In [1]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr tqdm -q
!pip install evaluate -q
!pip install pytorch_lightning -q
!pip install peft -q
#!pip install datasets==2.15.0
#!pip install peft==0.7.1
#!pip install bitsandbytes==0.41.3
#!pip install accelerate==0.25.0
#!pip install trl==0.7.7
#!pip install tqdm==4.66.1
#!pip install flash-attn==2.4.2
#!pip install git+https://github.com/lvwerra/trl.git -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 M

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!/usr/local/cuda/bin/nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Sun Feb 25 22:06:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              10W /  7

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from peft import LoraConfig, get_peft_model, TaskType, PeftModel, PeftConfig
import os
import string
import torch.nn as nn
import pandas as pd
import numpy as np
import evaluate
import matplotlib.pyplot as plt
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import get_linear_schedule_with_warmup, AdamW
from sklearn.model_selection import train_test_split
from pytorch_lightning.callbacks import ModelCheckpoint

from tqdm import tqdm
tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## **Gestion de base de donnée**

In [ ]:
dataset = pd.read_json('/content/drive/MyDrive/AH/airbus-helicopters-hackathon-toolkit/data/airbus_helicopters_train_set.json').T
open_data = pd.read_json('/content/drive/MyDrive/AH/airbus-helicopters-hackathon-toolkit/data/open_source_dataset.json').T

In [ ]:
#Concatenate the two datasets
df = pd.concat([dataset, open_data[["original_text", "reference_summary", "uid"]]])
df

,original_text,reference_summary,uid
train_sum01,These general Standard Conditions of Sale appl...,These terms and conditions apply as soon as th...,train_sum01
train_sum010,Each Party represents to the other as at the d...,Each Party represents that the other is not a ...,train_sum010
train_sum0100,"All living, travelling and accommodation expen...","Expenses relating to the travelling, living an...",train_sum0100
train_sum0101,"Unless otherwise specified in the Contract, th...","Unless otherwise specified in the Contract, th...",train_sum0101
train_sum0102,Reasonable insurance coverage of risks arising...,Reasonable insurance coverage of risks arising...,train_sum0102
...,...,...,...
tosdr417,third party vendors including google use cooki...,this service allows tracking via third party c...,tosdr417
tosdr418,the third parties that display advertisements ...,this service employs third party cookies but w...,tosdr418
tosdr419,we may make use of third party service provide...,third parties may be involved in operating the...,tosdr419
tosdr420,please be aware that we may transfer your info...,your data may be processed and stored anywhere...,tosdr420


## **Description de la base de donnée**

In [ ]:
# Dimensions du dataframe
print("Le nombre de lignes du dataframe :", df.shape[0])
print("Le nombre de colonnes du dataframe :", df.shape[1])

Le nombre de lignes du dataframe : 859
Le nombre de colonnes du dataframe : 3


In [ ]:
# Description des colonnes
print("Le nom des colonnes :", df.columns)

Le nom des colonnes : Index(['original_text', 'reference_summary', 'uid'], dtype='object')


In [ ]:
# Description des données
summary = df.describe().T.reset_index()
print("Description du dataframe : ")
print(summary)

Description du dataframe : 
               index count unique  \
0      original_text   859    856   
1  reference_summary   859    701   
2                uid   859    859   

                                                 top freq  
0  The Seller reserves the right to change the su...    2  
1  the service may use tracking pixels web beacon...   11  
2                                           tosdr421    1  


In [ ]:
# Vérifier s'il existe des colonnes avec des valeurs nulles dans le dataframe
colonnes_nulles = df.columns[df.isnull().any()].tolist()

if colonnes_nulles:
    print("Les colonnes suivantes contiennent des valeurs nulles :", colonnes_nulles)
else:
    print("Aucune colonne ne contient de valeurs nulles.")

Aucune colonne ne contient de valeurs nulles.


In [ ]:
# Fonction pour calculer le nombre de mots dans une cellule
def nombre_mots(cellule):
    mots = cellule.split()  # Séparation de la cellule en mots
    return len(mots)  # Retourne le nombre de mots

# Appliquer la fonction "nombre_mots" pour chaque colonne
df['len_reference_summary'] = df['reference_summary'].apply(nombre_mots)
df['len_original_text']  = df['original_text'].apply(nombre_mots)

# Afficher les résultats
print("Pour la colonne 'reference_summary':")
print("Moyenne du nombre de mots:", df['len_reference_summary'].mean())
print("Nombre minimum de mots:", df['len_reference_summary'].min())
print("Nombre maximum de mots:", df['len_reference_summary'].max())
print("\n-----------------------------------------------------")
print("\nPour la colonne 'original_text':")
print("Moyenne du nombre de mots:", df['len_original_text'].mean())
print("Nombre minimum de mots:", df['len_original_text'].min())
print("Nombre maximum de mots:", df['len_original_text'].max())

Pour la colonne 'reference_summary':
Moyenne du nombre de mots: 22.08498253783469
Nombre minimum de mots: 1
Nombre maximum de mots: 102

-----------------------------------------------------

Pour la colonne 'original_text':
Moyenne du nombre de mots: 78.74388824214202
Nombre minimum de mots: 7
Nombre maximum de mots: 1077


## **Définition du model**

In [ ]:
#Importation du modèle
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model_base = T5ForConditionalGeneration.from_pretrained(model_name, return_dict = True)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
#Introduction du modèle PEFT avec LoRa
lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

MODEL = get_peft_model(model_base,
                            lora_config)

In [ ]:
#Vue sur le nombre de paramètres selectionnés
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(MODEL))

trainable model parameters: 3538944
all model parameters: 226442496
percentage of trainable model parameters: 1.56%


## **Data Preprocessing**

In [ ]:
class AirLingoSummaryData(Dataset):

    def __init__(self, data: pd.DataFrame, tokenizer: T5Tokenizer, text_max_len: int=512, summary_max_len: int=128):

        self.data = data
        self.tokenizer = tokenizer
        self.text_max_len = text_max_len
        self.summary_max_len = summary_max_len

    def __len__(self):

        return len(self.data)

    def __getitem__(self, idx: int):

    #
        data_row = self.data.iloc[idx]
        original_text = data_row["original_text"]

        text_tokenizer = tokenizer(
            original_text,
            max_length=self.text_max_len,
            padding="max_length",
            truncation=True,
            add_special_tokens=True,
            return_attention_mask=True,
            return_tensors="pt")

        summary_tokenizer = tokenizer(
            data_row["reference_summary"],
            max_length=self.summary_max_len,
            padding="max_length",
            truncation=True,
            add_special_tokens=True,
            return_attention_mask=True,
            return_tensors="pt")

        labels = summary_tokenizer["input_ids"]
        labels[labels ==0] = -100
        return dict(
            original_text = original_text,
            summary=data_row["reference_summary"],
            text_input_ids = text_tokenizer["input_ids"].flatten(),
            text_attention_mask = text_tokenizer["attention_mask"].flatten(),
            labels = labels.flatten(),
            labels_attention_mask= summary_tokenizer["attention_mask"].flatten()
        )

In [ ]:
class AirLingoSummaryDataModule(pl.LightningDataModule):

    def __init__(
        self,
        train_df: pd.DataFrame,
        test_df: pd.DataFrame,
        tokenizer: T5Tokenizer,
        batch_size: int = 8,
        text_max_len: int=512,
        summary_max_len: int=128
    ):

        super().__init__()

        self.train_df = train_df
        self.test_df = test_df

        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.text_max_len = text_max_len
        self.summary_max_len = summary_max_len

    def setup(self, stage=None):
        self.train_dataset = AirLingoSummaryData(
            self.train_df,
            self.tokenizer,
            self.text_max_len,
            self.summary_max_len)

        self.test_dataset = AirLingoSummaryData(
            self.test_df,
            self.tokenizer,
            self.text_max_len,
            self.summary_max_len
            )

    def train_dataloader(self):

        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=4)

    def val_dataloader(self):

        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=4)

In [ ]:
#Découper la base de donnée en donnée de validation et d'entrainement
train_df, val_df = train_test_split(df, test_size=0.2, shuffle = True, random_state=42)
val_df = val_df.reset_index(drop=True)


In [ ]:
N_EPOCHS = 25
BATCH_SIZE = 10
data_module = AirLingoSummaryDataModule(train_df, val_df,tokenizer, batch_size = BATCH_SIZE)

## **Entrainement du modèle**

In [ ]:
class AirLingoSummaryModel(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.model = MODEL

    def forward(self, input_ids, attention_mask, decoder_attention_mask, labels=None):
        outputs = self.model(input_ids=input_ids,
                             attention_mask=attention_mask,
                             labels=labels,
                             decoder_attention_mask=decoder_attention_mask)

        return outputs.loss, outputs.logits

    def training_step(self, batch, batch_idx):
        input_ids = batch["text_input_ids"]
        attention_mask = batch["text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]

        loss, output = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels = labels,
            decoder_attention_mask = labels_attention_mask)

        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch["text_input_ids"]
        attention_mask = batch["text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]

        loss, output = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels = labels,
            decoder_attention_mask = labels_attention_mask)

        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=0.0001)
        return optimizer

In [ ]:
#model definition
model = AirLingoSummaryModel()

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath= "/content/drive/MyDrive/AH/model_sv",
    filename= "checkpoint",
    save_weights_only=True,
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"

)

logger = TensorBoardLogger("lightning_logs", name = "news-summary")

trainer = pl.Trainer(
    logger=logger,
    callbacks=checkpoint_callback,
    max_epochs=N_EPOCHS
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                  | Params
------------------------------------------------
0 | model | PeftModelForSeq2SeqLM | 226 M 
------------------------------------------------
3.5 M     Trainable params
222 M     Non-trainable params
226 M     Total params
905.770   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 69: 'val_loss' reached 1.76644 (best 1.76644), saving model to '/content/drive/MyDrive/AH/model_sv/checkpoint.ckpt' as top 1


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 22, global step 1587: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 23, global step 1656: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 24, global step 1725: 'val_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=25` reached.


In [ ]:
 #Prendre le meilleur modèle
news_summary_model = AirLingoSummaryModel.load_from_checkpoint(
    trainer.checkpoint_callback.best_model_path
)
news_summary_model.freeze()

In [ ]:
#Sauvegarder le modèle
news_summary_model.model.save_pretrained("/content/drive/MyDrive/AH/model_sv")
tokenizer.save_pretrained("/content/drive/MyDrive/AH/model_sv")
torch.save(news_summary_model.model.state_dict(), "/content/drive/MyDrive/AH/model_sv/pytorch_model.bin")

## Evaluation du modèle

In [4]:
#loader le modèle sauvegarder
model_test = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/AH/model_sv", return_dict = True).to(device)
tokenizer_test = T5Tokenizer.from_pretrained("/content/drive/MyDrive/AH/model_sv")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
#Importer la base de donnée test
test_set = pd.read_json('/content/drive/MyDrive/AH/test_set.json').T
test_set

,original_text,uid
test_sum01,Customer’s Orders shall be confirmed by the Se...,test_sum01
test_sum010,Copying and/or reproducing and/or communicatio...,test_sum010
test_sum011,In no event shall the Parties be liable for an...,test_sum011
test_sum012,A Party may terminate all or part of the Contr...,test_sum012
test_sum013,AH owns all foreground intellectual property a...,test_sum013
test_sum014,The content and the time schedule of the train...,test_sum014
test_sum015,"Training Means mean the Helicopter, the tools,...",test_sum015
test_sum016,The Seller reserves the right to cancel any tr...,test_sum016
test_sum017,The Customer shall be responsible for loss of ...,test_sum017
test_sum018,"The Customer will be asked to fill, sign and r...",test_sum018


In [6]:
from datasets import Dataset
df_test  = Dataset.from_pandas(test_set)
df_test = df_test.remove_columns(['__index_level_0__'])
df_test

Dataset({
    features: ['original_text', 'uid'],
    num_rows: 50
})

In [9]:
#Faire de l'inférence sur le meilleur model
def summarize(text):
    inputs = tokenizer_test(text,
                       max_length=512,
                       truncation=True,
                       padding="max_length",
                       return_attention_mask = True,
                       add_special_tokens=True,
                       return_tensors="pt").to(device)
    summarized_ids = model_test.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=128,
        num_beams=5,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
      ).to(device)

    return " ".join([tokenizer_test.decode(token_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True) for token_ids in summarized_ids])

In [13]:
rouge = evaluate.load("rouge")

In [10]:
original_text = []
reference_summary = []
predicted_summary = []

for i in range(len(test_set)):

    text=test_set.iloc[i]["original_text"]
    original_text.append(test_set.iloc[i]["original_text"])
    predicted_summary.append(summarize(text))

In [11]:
predicted_summary

['Customer’s Orders shall be confirmed by the Seller in writing. The delivery schedule becomes effective upon receipt of the down-payment.',
 'Copying and/or reproducing or communication and/or transmission to a third party without the Seller’s express approval is strictly forbidden.',
 'In no event shall the Parties be liable for any damages.',
 'A Party may terminate all or part of the Contract for default of the other Party in the event that the other Party persistently fails to perform its obligations.',
 'AH owns all foreground intellectual property and know-how generated by the analysis of the Data.',
 'The content and the time schedule of the training courses shall be confirmed at the time of signing of the Contract, at Customer’s request and depending on Seller’s instructor, Helicopter and/or simulator availability.',
 'Helicopter, tools, simulator, infrastructure and equipment used by the Seller to perform the Training Service(s).',
 'The Seller reserves the right to cancel an

In [14]:
rouge_score = rouge.compute(
    predictions=predicted_summary,
    references = test_set.iloc[0:len(test_set)]["original_text"],
    use_aggregator = True,
    use_stemmer=True,

)
rouge_score

{'rouge1': 0.7256698528617255,
 'rouge2': 0.6756296943691886,
 'rougeL': 0.7251283253268708,
 'rougeLsum': 0.7241912204344448}

In [15]:
gen_sum = pd.DataFrame({"generated_summary":predicted_summary,
                       "uid":test_set["uid"]})
gen_sum

,generated_summary,uid
test_sum01,Customer’s Orders shall be confirmed by the Se...,test_sum01
test_sum010,Copying and/or reproducing or communication an...,test_sum010
test_sum011,In no event shall the Parties be liable for an...,test_sum011
test_sum012,A Party may terminate all or part of the Contr...,test_sum012
test_sum013,AH owns all foreground intellectual property a...,test_sum013
test_sum014,The content and the time schedule of the train...,test_sum014
test_sum015,"Helicopter, tools, simulator, infrastructure a...",test_sum015
test_sum016,The Seller reserves the right to cancel any tr...,test_sum016
test_sum017,The Customer shall be responsible for loss and...,test_sum017
test_sum018,"Customer will be asked to fill, sign and retur...",test_sum018


In [20]:
json_data = gen_sum.to_dict(orient='index')
json_data

{'test_sum01': {'generated_summary': 'Customer’s Orders shall be confirmed by the Seller in writing. The delivery schedule becomes effective upon receipt of the down-payment.',
  'uid': 'test_sum01'},
 'test_sum010': {'generated_summary': 'Copying and/or reproducing or communication and/or transmission to a third party without the Seller’s express approval is strictly forbidden.',
  'uid': 'test_sum010'},
 'test_sum011': {'generated_summary': 'In no event shall the Parties be liable for any damages.',
  'uid': 'test_sum011'},
 'test_sum012': {'generated_summary': 'A Party may terminate all or part of the Contract for default of the other Party in the event that the other Party persistently fails to perform its obligations.',
  'uid': 'test_sum012'},
 'test_sum013': {'generated_summary': 'AH owns all foreground intellectual property and know-how generated by the analysis of the Data.',
  'uid': 'test_sum013'},
 'test_sum014': {'generated_summary': 'The content and the time schedule of t

In [21]:

import json
with open("/content/drive/MyDrive/AH/generated_dataset.json", 'w') as json_file:
    json.dump(json_data, json_file, indent=4)

In [22]:
len(json_data)

50

In [ ]:
print("==="*10)
print(test_set.iloc[2]["original_text"])
print("==="*10)
print(predicted_summary[2])

In no event shall the Parties be liable for any indirect, consequential, incidental, special or punitive damages of any kind, including, but not limited to, damages for any loss of use or profit, loss of assets, loss of asset value, loss resulting from business disruption, loss of goodwill or loss of contractual opportunity by the other Party.
In no event shall the Parties be liable for any damages.


In [66]:
import json

ref_data = pd.read_json('/content/drive/MyDrive/AH/evaluation-toolkit/reference_dataset.json', 'r').T
ref_data.rename(columns={'original_text': 'reference_summary'}, inplace=True)
ref_data

,reference_summary,uid
test_sum01,Customer’s Orders shall be confirmed by the Se...,test_sum01
test_sum010,Copying and/or reproducing and/or communicatio...,test_sum010
test_sum011,In no event shall the Parties be liable for an...,test_sum011
test_sum012,A Party may terminate all or part of the Contr...,test_sum012
test_sum013,AH owns all foreground intellectual property a...,test_sum013
test_sum014,The content and the time schedule of the train...,test_sum014
test_sum015,"Training Means mean the Helicopter, the tools,...",test_sum015
test_sum016,The Seller reserves the right to cancel any tr...,test_sum016
test_sum017,The Customer shall be responsible for loss of ...,test_sum017
test_sum018,"The Customer will be asked to fill, sign and r...",test_sum018


In [67]:
ref_data = ref_data.to_dict(orient='index')
ref_data

{'test_sum01': {'reference_summary': 'Customer’s Orders shall be confirmed by the Seller in writing. The Contract shall become binding upon receipt by the Customer of the Seller’s Order Confirmation and the delivery schedule shall become effective upon receipt by the Seller of the down-payment when relevant (as mentioned under article 6.2) and subject to compliance by the Customer to article 4.1.',
  'uid': 'test_sum01'},
 'test_sum010': {'reference_summary': 'Copying and/or reproducing and/or communication and/or transmission to a third party of the Products or Services or technical information or Technical Data or training manuals, either wholly or partially, without the Seller’s written express approval is strictly forbidden (except for the copying by the Customer of technical documentation provided by the Seller exclusively for the purposes of operation and maintenance of the Helicopters by the Customer).',
  'uid': 'test_sum010'},
 'test_sum011': {'reference_summary': 'In no event